In [1]:
!pip install datasets transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s e

In [2]:
# Loading the Stanford Question Answering Dataset (SQuAD)
from datasets import load_dataset
dataset = load_dataset("rajpurkar/squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
# Getting the training and the validation dataset
train_dataset = dataset["train"].select(range(1000))

In [4]:
train_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [5]:
train_dataset.num_rows

1000

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()[0]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
import chromadb

# Crée une instance du client ChromaDB
client = chromadb.Client()

# Crée une nouvelle collection appelée 'squad_contexts' (si elle n'existe pas)
collection = client.create_collection(name="squad_contexts")

In [8]:
batch_size = 32
batch_embeddings = []
batch_contexts = []
batch_metadatas = []
batch_ids = []  # To store unique IDs
count_batch = 0
len_batches = len(train_dataset) // batch_size + 1

for i, example in enumerate(train_dataset):
    context = example["context"]
    context_embedding = embed_text(context)

    # Convert the numpy ndarray embedding to a list of floats
    batch_embeddings.append(context_embedding.tolist())
    batch_contexts.append(context)
    batch_metadatas.append({"id": example["id"]})
    batch_ids.append(example["id"])  # Add the unique ID to the batch

    # Add data in batches of `batch_size`
    if (i + 1) % batch_size == 0 or (i + 1) == len(train_dataset):
        count_batch += 1
        print(f"Batch {count_batch}/{len_batches}")

        # Pass 'ids' argument along with other arguments
        collection.add(
            embeddings=batch_embeddings,
            documents=batch_contexts,
            metadatas=batch_metadatas,
            ids=batch_ids  # Add the list of IDs here
        )

        # Reset the lists after adding
        batch_embeddings = []
        batch_contexts = []
        batch_metadatas = []
        batch_ids = []  # Reset the IDs list


Batch 1/32
Batch 2/32
Batch 3/32
Batch 4/32
Batch 5/32
Batch 6/32
Batch 7/32
Batch 8/32
Batch 9/32
Batch 10/32
Batch 11/32
Batch 12/32
Batch 13/32
Batch 14/32
Batch 15/32
Batch 16/32
Batch 17/32
Batch 18/32
Batch 19/32
Batch 20/32
Batch 21/32
Batch 22/32
Batch 23/32
Batch 24/32
Batch 25/32
Batch 26/32
Batch 27/32
Batch 28/32
Batch 29/32
Batch 30/32
Batch 31/32
Batch 32/32


In [9]:
def search_query(query, n_results=1):
    query_embedding = embed_text(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return results

In [10]:
# Exemple de recherche
query = "What is the capital of France?"
results = search_query(query)

# Affichage des résultats
for idx, result in enumerate(results['documents'][0]):
    print(f"Rank {idx + 1}: {result}")


Rank 1: As of 2012[update] research continued in many fields. The university president, John Jenkins, described his hope that Notre Dame would become "one of the pre–eminent research institutions in the world" in his inaugural address. The university has many multi-disciplinary institutes devoted to research in varying fields, including the Medieval Institute, the Kellogg Institute for International Studies, the Kroc Institute for International Peace studies, and the Center for Social Concerns. Recent research includes work on family conflict and child development, genome mapping, the increasing trade deficit of the United States with China, studies in fluid mechanics, computational science and engineering, and marketing trends on the Internet. As of 2013, the university is home to the Notre Dame Global Adaptation Index which ranks countries annually based on how vulnerable they are to climate change and how prepared they are to adapt.


In [37]:
def evaluate_search_engine():
    total_precision = 0
    n_questions = len(train_dataset)

    for example in train_dataset:
        question = example["question"]
        true_answer = example["answers"]["text"][0]

        # Debug: Print the question and true answer
        #print(f"Question: {question}")
        #print(f"True Answer: {true_answer}")

        # Generate the embedding for the question
        query_embedding = embed_text(question)

        # Perform search in ChromaDB
        results = collection.query(query_embeddings=[query_embedding], n_results=20)

        # Debug: Print out retrieved documents
        retrieved_answers = results['documents'][0]
        #rint(f"Retrieved Answers: {retrieved_answers}")

        # Check if any retrieved passage contains the true answer
        relevant = any(true_answer in passage for passage in retrieved_answers)
        total_precision += int(relevant)

    # Calculate metrics
    precision = total_precision / n_questions

    return precision


In [40]:
precision = evaluate_search_engine()
print(f"Précision: {precision:.4f}")

Précision: 0.4620
